<a href="https://colab.research.google.com/github/VGGatGitHub/Summer2021/blob/main/QISKit_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook experimenting how to use DOcplex and IBM Quantum.

# 0. Install libraries & clone repo

In [ ]:
import sys

if hasattr(sys, 'real_prefix'):
    #we are in a virtual env.
    !pip install cplex && pip install docplex
else:
    !pip install --user cplex && pip install --user docplex

print()
!pip show cplex
print()
!pip show docplex

#it my be useful to retstart you runtime

In [ ]:
!pip install qiskit
!pip show qiskit

# !pip install qiskit-optimization
#!pip show qiskit-optimization

#it my be useful to retstart you runtime

**Possibly need to restart runtime here.**

In [ ]:
#it my be useful to retstart you runtime

# clone repo
!git clone https://github.com/VGGatGitHub/Summer2021.git

# 1. Load Data

In [1]:
#next run the model.py in the folder introScenario
!pwd
%cd ./Summer2021/introScenario/
#./Summer2021/intermediateScenario
#./Summer2021/introScenario/
#%ls


/content
/content/Summer2021/introScenario


In [2]:
import pandas as pd

# supply + costs
supply_file = './plants.csv'

# demand
demand_file = './customerDemand.csv'

# for now load example data (plants.csv and customerDemand.csv under introScenario folder)
s_df = pd.read_csv(supply_file)
d_df = pd.read_csv(demand_file)

In [3]:
d_df

,Product,Demand
0,handSanitizer,100
1,mask,120


In [4]:
s_df

,Plants,Cost,Capacity,Product
0,1,3,40,mask
1,2,2,30,mask
2,3,1,30,handSanitizer
3,4,3,100,handSanitizer
4,5,2,60,mask
5,6,1,45,mask


In [5]:
#s_df.drop([1,2,3], inplace = True)
s_df

,Plants,Cost,Capacity,Product
0,1,3,40,mask
1,2,2,30,mask
2,3,1,30,handSanitizer
3,4,3,100,handSanitizer
4,5,2,60,mask
5,6,1,45,mask


In [6]:
inputs={'customerDemand':d_df,'plants':s_df}

# 2. Solve the DO problem via DOcplex

In [7]:
#If you get an error you may have to restart the runtime
import cplex
import docplex.mp
from docplex.mp.model import Model

In [8]:
global output_lock
#with output_lock:

try:  
  if output_lock:
    print(output_lock)
except:
  output_lock=dict({'output_lock':{}})

try: 
  if outputs:
    print(outputs)
except:
  outputs=dict()


In [9]:
exec(open("./model.py").read())

* building wado model
model building done in 0.20445871353149414 secs
* running wado model
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               120
Found incumbent of value 0.000000 after 0.02 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 20 rows and 12 columns.
All rows and columns eliminated.
Presolve time = 0.02 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.06 sec. (0.01 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.06 sec. (0.01 ticks)
Solve status: JobSolveStatus.OPTIMAL_SOLUTION
* model docplex_model1 solved with objective = -105680.000
*  KPI: total plants allocations                  = 112640.000
*  KPI: total Cost of plants over all all

In [10]:
outputs


{'plantsAllocation':    plants  ...  plants minValueAllocationForAssignment
 0       1  ...                                       0
 1       2  ...                                       0
 2       3  ...                                       0
 3       4  ...                                       0
 4       5  ...                                       0
 5       6  ...                                       0
 
 [6 rows x 7 columns]}

In [11]:
outputs['plantsAllocation']

,plants,plants allocation decision,plants selection decision,plants Capacity,plants Product,plants Cost,plants minValueAllocationForAssignment
0,1,0.0,1.0,40,mask,3,0
1,2,15.0,1.0,30,mask,2,0
2,3,30.0,1.0,30,handSanitizer,1,0
3,4,70.0,1.0,100,handSanitizer,3,0
4,5,60.0,1.0,60,mask,2,0
5,6,45.0,1.0,45,mask,1,0


In [12]:
model

docplex.mp.Model['docplex_model1']

In [13]:
solution=model.solve()
print(solution)
solution

solution for: docplex_model1
objective: -105680
x2=15
x3=30
x4=70
x5=60
x6=45
x7=1
x8=1
x9=1
x10=1
x11=1
x12=1



docplex.mp.solution.SolveSolution(obj=-105680,values={x2:15,x3:30,x4:70,..

In [14]:
cost = solution.get_objective_value()
x = solution.get_all_values()
x = np.array(x)
print(cost,x)


-105680.0 [ 0. 15. 30. 70. 60. 45.  1.  1.  1.  1.  1.  1.]


#Minimum Eigen Optimizers: exact, qaoa, rqaoa

In [15]:
#https://quantum-computing.ibm.com/lab/docs/iql/optimization

from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

aqua_globals.random_seed = 10598
#('qasm_simulator') #qasm_simulator or statevector_simulator

quantum_instance = QuantumInstance(BasicAer.get_backend('qasm_simulator'),
                                   seed_simulator=aqua_globals.random_seed,
                                   seed_transpiler=aqua_globals.random_seed)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])
exact_mes = NumPyMinimumEigensolver()

qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver
rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=1, min_num_vars_optimizer=exact)


/usr/local/lib/python3.7/dist-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/usr/local/lib/python3.7/dist-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: The variable qiskit.aqua.aqua_globals is deprecated. It was moved/refactored to qiskit.utils.algorithm_globals (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/

In [16]:
# create a QUBO
if False: #qp_eq_bin or qp_eq
  qubo=qp_eq_bin
  print(mod.export_as_lp_string())
else:
  qubo = QuadraticProgram()
  qubo.binary_var('x')
  qubo.binary_var('y')
  qubo.binary_var('z')
  qubo.minimize(linear=[1,-2,3], quadratic={('x', 'y'): 1, ('x', 'z'): -1, ('y', 'z'): 2})
  print(qubo.export_as_lp_string())


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



In [17]:
%time rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/operators/operator_base.py:46: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.operators', 'qiskit.opflow', 'qiskit-terra')
/usr/local/lib/python3.7/dist-packages/qiskit/aqua/components/variational_forms/variational_form.py:48: DeprecationWarning: The package qiskit.aqua.components.variational_forms is deprecated. For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.components.variational_forms')


CPU times: user 654 ms, sys: 125 ms, total: 780 ms
Wall time: 2.12 s
optimal function value: 1.0
optimal value: [1. 0. 0.]
status: SUCCESS


In [18]:
%time qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

CPU times: user 185 ms, sys: 4.32 ms, total: 189 ms
Wall time: 189 ms
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


In [19]:
%time exact_result = exact.solve(qubo)
print(exact_result)

CPU times: user 25 ms, sys: 0 ns, total: 25 ms
Wall time: 25.4 ms
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


In [20]:
op, offset = qubo.to_ising()
qp=QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.export_as_lp_string())


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x_0 - 2 x_1 + 3 x_2 + [ 2 x_0*x_1 - 2 x_0*x_2 + 4 x_1*x_2 ]/2
Subject To

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1

Binaries
 x_0 x_1 x_2
End



In [21]:
%time exact_result = exact.solve(qp)
print(exact_result)

CPU times: user 22.1 ms, sys: 1.14 ms, total: 23.2 ms
Wall time: 23.5 ms
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


#Try Grover Optimizer: grover_optimizer

In [22]:
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram
from qiskit import BasicAer
from docplex.mp.model import Model

backend = BasicAer.get_backend('statevector_simulator') #qasm_simulator or statevector_simulator
grover_optimizer = GroverOptimizer(6, num_iterations=10, quantum_instance=backend)


In [23]:
%time results = grover_optimizer.solve(qp)#qubo or qp
print("x={}".format(results.x))
print("fval={}".format(results.fval))

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/algorithms/amplitude_amplifiers/grover.py:216: DeprecationWarning: The package qiskit.aqua.algorithms.amplitude_amplifiers is deprecated. It was moved/refactored to qiskit.algorithms.amplitude_amplifiers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  'qiskit.algorithms.amplitude_amplifiers', 'qiskit-terra')


CPU times: user 52.7 s, sys: 1.95 s, total: 54.7 s
Wall time: 50.9 s
x=[0. 1. 0.]
fval=-2.0


In [24]:
%time results = grover_optimizer.solve(qubo)#qubo or qp
print("x={}".format(results.x))
print("fval={}".format(results.fval))

CPU times: user 59.4 s, sys: 2.26 s, total: 1min 1s
Wall time: 57.5 s
x=[0. 1. 0.]
fval=-2.0


#Try ADMM Optimizer (using cobyla or cplex)


In [25]:
import time
from typing import List, Optional, Any
import numpy as np
import matplotlib.pyplot as plt 

from docplex.mp.model import Model

from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import CobylaOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram
from qiskit.optimization.algorithms.admm_optimizer import ADMMParameters, ADMMOptimizer

# If CPLEX is installed, you can uncomment this line to import the CplexOptimizer.
# CPLEX can be used in this tutorial to solve the convex continuous problem, 
# but also as a reference to solve the QUBO, or even the full problem.
# 
from qiskit.optimization.algorithms import CplexOptimizer

In [26]:
# define COBYLA optimizer to handle convex continuous problems.
cobyla = CobylaOptimizer()

# define QAOA via the minimum eigen optimizer
qaoa = MinimumEigenOptimizer(QAOA(quantum_instance=BasicAer.get_backend('qasm_simulator')))
#('qasm_simulator') #qasm_simulator or statevector_simulator
# exact QUBO solver as classical benchmark
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver()) # to solve QUBOs


In [27]:
print(qubo)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



In [28]:
admm_params = ADMMParameters(
                            rho_initial=1001, 
                            beta=1000, 
                            factor_c=900,
                            maxiter=100,
                            three_block=True, tol=1.e-6
                        )

In [29]:
# in case CPLEX is installed it can also be used for the convex problems, the QUBO, 
# or as a benchmark for the full problem.
#
cplex = CplexOptimizer()

# define QUBO optimizer
# qubo_optimizer = exact #qaoa #rqaoa #grover_optimizer

# qubo_optimizer = cplex  # uncomment to use CPLEX instead

i=-1
optms=['cplex','exact','qaoa','rqaoa']
for qubo_optimizer in [cplex,exact,qaoa,rqaoa]:

  # define classical optimizer
  convex_optimizer = cobyla
  #convex_optimizer = cplex  # uncomment to use CPLEX instead

  # initialize ADMM with classical QUBO and convex optimizer
  admm = ADMMOptimizer(params=admm_params, 
                      qubo_optimizer=qubo_optimizer,
                      continuous_optimizer=convex_optimizer)

  # run ADMM to solve problem
  i+=1
  print("\nOptimizer:",optms[i])
  %time result = admm.solve(qubo)
  print("x={}".format(result.x))
  print("fval={:.2f}".format(result.fval))


Optimizer: cplex
CPU times: user 10.4 s, sys: 4.03 s, total: 14.5 s
Wall time: 17.7 s
x=[0. 0. 0.]
fval=0.00

Optimizer: exact
CPU times: user 11.4 s, sys: 3.24 s, total: 14.7 s
Wall time: 17.2 s
x=[0. 0. 0.]
fval=0.00

Optimizer: qaoa
CPU times: user 30.3 s, sys: 3.87 s, total: 34.2 s
Wall time: 36.8 s
x=[0. 0. 0.]
fval=0.00

Optimizer: rqaoa
CPU times: user 41.1 s, sys: 3.77 s, total: 44.8 s
Wall time: 47.3 s
x=[1. 0. 0.]
fval=1.00


In [30]:
# run ADMM to solve problem
%time result = admm.solve(qp,)
print("x={}".format(result.x))
print("fval={:.2f}".format(result.fval))

CPU times: user 42 s, sys: 3.87 s, total: 45.9 s
Wall time: 48.5 s
x=[1. 0. 0.]
fval=1.00


#Minimum Eigen Optimizer using VQE

In [31]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.aqua.operators import PauliExpectation, CVaRExpectation
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import LinearEqualityToPenalty
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit import execute, Aer
from qiskit.aqua import aqua_globals

import numpy as np
import matplotlib.pyplot as plt
from docplex.mp.model import Model

In [32]:
aqua_globals.random_seed = 123456
n = 12            # number of assets for the variational ansatz

In [33]:
# set classical optimizer
maxiter = 100
optimizer = COBYLA(maxiter=maxiter)

# set variational ansatz
var_form = RealAmplitudes(n, reps=1)
m = var_form.num_parameters

# set backend
backend_name = 'qasm_simulator'  # use this for QASM simulator qasm_simulator or statevector_simulator
# backend_name = 'statevector_simulator'  # use this for statevector simlator
backend = Aer.get_backend(backend_name)  

# run variational optimization for different values of alpha
alphas = [1.0, 0.50, 0.25]  # confidence levels to be evaluated

In [34]:
# dictionaries to store optimization progress and results
objectives = {alpha: [] for alpha in alphas}  # set of tested objective functions w.r.t. alpha
results = {}  # results of minimum eigensolver w.r.t alpha

# callback to store intermediate results
def callback(i, params, obj, stddev, alpha):
    # we translate the objective from the internal Ising representation
    # to the original optimization problem
    objectives[alpha] += [-(obj + offset)]  

# loop over all given alpha values
for alpha in alphas:
    
    # initialize CVaR_alpha objective
    cvar_exp = CVaRExpectation(alpha, PauliExpectation())
    cvar_exp.compute_variance = lambda x: [0]  # to be fixed in PR #1373
    
    # initialize VQE using CVaR
    vqe = VQE(expectation=cvar_exp, optimizer=optimizer, var_form=var_form, quantum_instance=backend,
              callback=lambda i, params, obj, stddev: callback(i, params, obj, stddev, alpha))
   
    # initialize optimization algorithm based on CVaR-VQE
    opt_alg = MinimumEigenOptimizer(vqe)

    # solve problem
    print()
    %time results[alpha] = opt_alg.solve(qubo) #VGG qp, qubo, mod, mod_Qubo
    
    # print results
    print('alpha = {}:'.format(alpha))
    print(results[alpha])
  


CPU times: user 858 ms, sys: 16.8 ms, total: 875 ms
Wall time: 879 ms
alpha = 1.0:
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS

CPU times: user 609 ms, sys: 9.03 ms, total: 618 ms
Wall time: 618 ms
alpha = 0.5:
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS

CPU times: user 637 ms, sys: 4.56 ms, total: 641 ms
Wall time: 648 ms
alpha = 0.25:
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


# 3. Solve the problem using QISKit

In [54]:
# take a look at https://quantum-computing.ibm.com/lab/docs/iql/optimization

#from qiskit_optimization import QuadraticProgram
#from qiskit_optimization.translators import from_docplex_mp

from qiskit.optimization import QuadraticProgram

# load from a Docplex model
mod = QuadraticProgram()
mod.from_docplex(model)
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - 464 x0 - 480 x1 - 496 x2 - 464 x3 - 480 x4 - 496 x5
Subject To
 _L_EXPR_1: x0 - 40 x6 <= 0
 _L_EXPR_2: x1 - 30 x7 <= 0
 _L_EXPR_3: x2 - 30 x8 <= 0
 _L_EXPR_4: x3 - 100 x9 <= 0
 _L_EXPR_5: x4 - 60 x10 <= 0
 _L_EXPR_6: x5 - 45 x11 <= 0
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_13: x0 <= 40
 _L_EXPR_14: x1 <= 30
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_17: x4 <= 60
 _L_EXPR_18: x5 <= 45
 _L_EXPR_19: x2 + x3 <= 100
 _L_EXPR_20: x0 + x1 + x4 + x5 <= 120

Bounds
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1

Binaries
 x6 x7 x8 x9 x10 x11

Generals
 x0 x1 x2 x3 x4 x5
End



In [55]:
%time solution = model.solve()
print(solution)
cost = solution.get_objective_value()
x = solution.get_all_values()
x = np.array(x)
print(cost,x)


CPU times: user 5.55 ms, sys: 0 ns, total: 5.55 ms
Wall time: 9.3 ms
solution for: docplex_model1
objective: -105680
x2=15
x3=30
x4=70
x5=60
x6=45
x7=1
x8=1
x9=1
x10=1
x11=1
x12=1

-105680.0 [ 0. 15. 30. 70. 60. 45.  1.  1.  1.  1.  1.  1.]


In [56]:
for eq_n in range(19,21):
  lc_name='_L_EXPR_'+str(eq_n)
  lin_c = mod.get_linear_constraint(lc_name)
  print('lin_c:',lc_name,lin_c.linear.to_dict(use_name=True), lin_c.sense, lin_c.rhs)
  mod.remove_linear_constraint(lc_name)
  mod.linear_constraint(lin_c.linear.to_dict(use_name=True), sense='==', rhs=lin_c.rhs, name=lc_name)

print(mod.export_as_lp_string())


lin_c: _L_EXPR_19 {'x2': 1.0, 'x3': 1.0} ConstraintSense.LE 100
lin_c: _L_EXPR_20 {'x0': 1.0, 'x1': 1.0, 'x4': 1.0, 'x5': 1.0} ConstraintSense.LE 120
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - 464 x0 - 480 x1 - 496 x2 - 464 x3 - 480 x4 - 496 x5
Subject To
 _L_EXPR_1: x0 - 40 x6 <= 0
 _L_EXPR_2: x1 - 30 x7 <= 0
 _L_EXPR_3: x2 - 30 x8 <= 0
 _L_EXPR_4: x3 - 100 x9 <= 0
 _L_EXPR_5: x4 - 60 x10 <= 0
 _L_EXPR_6: x5 - 45 x11 <= 0
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_13: x0 <= 40
 _L_EXPR_14: x1 <= 30
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_17: x4 <= 60
 _L_EXPR_18: x5 <= 45
 _L_EXPR_19: x2 + x3 = 100
 _L_EXPR_20: x0 + x1 + x4 + x5 = 120

Bounds
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1

Binaries
 x6 x7 x8 x9 x10 x11

Generals
 x0 x1 x2 x3 x4 x5
End



In [57]:
for eq_n in range(13,19):
  lc_name='_L_EXPR_'+str(eq_n)
  mod.remove_linear_constraint(lc_name)

print(mod.export_as_lp_string())


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - 464 x0 - 480 x1 - 496 x2 - 464 x3 - 480 x4 - 496 x5
Subject To
 _L_EXPR_1: x0 - 40 x6 <= 0
 _L_EXPR_2: x1 - 30 x7 <= 0
 _L_EXPR_3: x2 - 30 x8 <= 0
 _L_EXPR_4: x3 - 100 x9 <= 0
 _L_EXPR_5: x4 - 60 x10 <= 0
 _L_EXPR_6: x5 - 45 x11 <= 0
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_19: x2 + x3 = 100
 _L_EXPR_20: x0 + x1 + x4 + x5 = 120

Bounds
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1

Binaries
 x6 x7 x8 x9 x10 x11

Generals
 x0 x1 x2 x3 x4 x5
End



In [60]:
constant =mod.objective.constant
linear=s_df['Cost'].values
quadratic=np.kron(linear.reshape(1,-1),linear.reshape(-1,1))
#linear=mod.objective.linear.to_array()
mod.minimize(constant=constant, linear=linear, quadratic=quadratic)
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: 3 x0 + 2 x1 + x2 + 3 x3 + 2 x4 + x5 + [ 18 x0^2 + 24 x0*x1 + 12 x0*x2
      + 36 x0*x3 + 24 x0*x4 + 12 x0*x5 + 8 x1^2 + 8 x1*x2 + 24 x1*x3 + 16 x1*x4
      + 8 x1*x5 + 2 x2^2 + 12 x2*x3 + 8 x2*x4 + 4 x2*x5 + 18 x3^2 + 24 x3*x4
      + 12 x3*x5 + 8 x4^2 + 8 x4*x5 + 2 x5^2 ]/2
Subject To
 _L_EXPR_1: x0 - 40 x6 <= 0
 _L_EXPR_2: x1 - 30 x7 <= 0
 _L_EXPR_3: x2 - 30 x8 <= 0
 _L_EXPR_4: x3 - 100 x9 <= 0
 _L_EXPR_5: x4 - 60 x10 <= 0
 _L_EXPR_6: x5 - 45 x11 <= 0
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_19: x2 + x3 = 100
 _L_EXPR_20: x0 + x1 + x4 + x5 = 120

Bounds
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1

Binaries
 x6 x7 x8 x9 x10 x11

Generals
 x0 x1 x2 x3 x4 x5
End



In [61]:

print('\n',model.solve())

solution=cplex.solve(mod)
print(solution)



 solution for: docplex_model1
objective: -105680
x2=15
x3=30
x4=70
x5=60
x6=45
x7=1
x8=1
x9=1
x10=1
x11=1
x12=1

optimal function value: 189660.0
optimal value: [-0. 30. 30. 70. 45. 45.  1.  1.  1.  1.  1.  1.]
status: INFEASIBLE


In [ ]:
print('constant:\t\t\t', mod.objective.constant)

print('linear dict:\t\t\t', mod.objective.linear.to_dict())
print('linear array:\t\t\t', mod.objective.linear.to_array())
print('linear array as sparse matrix:\n', mod.objective.linear.coefficients, '\n')

print('quadratic dict w/ index:\t', mod.objective.quadratic.to_dict())
print('quadratic dict w/ name:\t\t', mod.objective.quadratic.to_dict(use_name=True))
print('symmetric quadratic dict w/ name:\t', mod.objective.quadratic.to_dict(use_name=True, symmetric=True))
print('quadratic matrix:\n', mod.objective.quadratic.to_array(),'\n')
print('symmetric quadratic matrix:\n', mod.objective.quadratic.to_array(symmetric=True),'\n')
print('quadratic matrix as sparse matrix:\n', mod.objective.quadratic.coefficients)


In [62]:
from qiskit.optimization.converters import QuadraticProgramToQubo
conv = QuadraticProgramToQubo()
mod_Qubo = conv.convert(mod)


In [ ]:
len(mod_Qubo._variables_index)

894

In [ ]:
print('constant:\t\t\t', mod_Qubo.objective.constant)
print('linear array as sparse matrix:\n', mod_Qubo.objective.linear.coefficients, '\n')
print('quadratic matrix as sparse matrix:\n', mod_Qubo.objective.quadratic.coefficients)

In [ ]:
from qiskit.optimization.converters import InequalityToEquality
from qiskit.optimization.converters import IntegerToBinary
from qiskit.optimization.converters import LinearEqualityToPenalty

ineq2eq = InequalityToEquality()
qp_eq = ineq2eq.convert(mod)
#print(qp_eq.export_as_lp_string())

int2bin = IntegerToBinary()
qp_eq_bin = int2bin.convert(qp_eq)
#print(qp_eq_bin.export_as_lp_string())

lineq2penalty = LinearEqualityToPenalty()
mod_Qubo2 = lineq2penalty.convert(qp_eq_bin)
#print(qubo.export_as_lp_string())


In [ ]:
len(mod_Qubo2._variables_index)

894

In [ ]:
""" it takes too long to do the next transfirmation 
op, offset = mod_Qubo.to_ising()
QPI=QuadraticProgram()
QPI.from_ising(op, offset, linear=True)
"""

In [ ]:
"""
print('constant:\t\t\t', QPI.objective.constant)
print('linear array as sparse matrix:\n', QPI.objective.linear.coefficients, '\n')
print('quadratic matrix as sparse matrix:\n', QPI.objective.quadratic.coefficients)
"""

In [ ]:
%time results = grover_optimizer.solve(mod_Qubo)#qubo or qp
print("x={}".format(results.x))
print("fval={}".format(results.fval))

In [ ]:
# solve problem using VQE
%time results[alpha] = opt_alg.solve(mod_Qubo) #VGG qp, qubo, mod, mod_Qubo
# print results
print('alpha = {}:'.format(alpha))
print(results[alpha])

In [ ]:
#exact, qaoa, rqaoa, grover_optimizer, admm
%time result = qaoa.solve(mod_Qubo) #rqaoa, grover_optimizer run out of memory after 28min
print(result)
#statevector_simulator with 402 qubits for admm.solve(mod) and admm_rqaoa had filed after 4m